# GPU-Jupyter

This Jupyterlab Instance is connected to the GPU via CUDA drivers. In this notebook, we test the installation and perform some basic operations on the GPU.

## Test GPU connection

#### Using the following command, your GPU type and its NVIDIA-SMI driver version should be listed:

In [1]:
!nvidia-smi

Thu Mar 17 15:40:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.00       Driver Version: 515.36       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   53C    P0    50W / 250W |   1705MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Now, test if PyTorch can access the GPU via CUDA:

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(tf.config.list_physical_devices('XLA_GPU'))
device_lib.list_local_devices()

2022-03-17 15:40:49.308573: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-17 15:40:49.337677: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-17 15:40:49.338006: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[]


2022-03-17 15:40:49.339022: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-17 15:40:49.340468: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-17 15:40:49.340850: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-17 15:40:49.341173: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built witho

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16871833971725379527,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6221463552
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13801559836489076464
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"]

In [4]:
from __future__ import print_function
import numpy as np
import torch
a = torch.rand(5, 3)
a

tensor([[0.2957, 0.4422, 0.7342],
        [0.6745, 0.7849, 0.1435],
        [0.3412, 0.7302, 0.4281],
        [0.6303, 0.2907, 0.8923],
        [0.7190, 0.5049, 0.4552]])

## Performance test

#### Now we want to know how much faster a typical operation is using GPU. Therefore we do the same operation in numpy, PyTorch and PyTorch with CUDA. The test operation is the calculation of the prediction matrix that is done in a linear regression.

### 1) Numpy

In [5]:
x = np.random.rand(10000, 256)

In [6]:
%%timeit
H = x.dot(np.linalg.inv(x.transpose().dot(x))).dot(x.transpose())

278 ms ± 31.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2) PyTorch

In [7]:
x = torch.rand(10000, 256)

In [8]:
%%timeit
# Calculate the projection matrix of x on the CPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

87.6 ms ± 4.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 3) PyTorch on GPU via CUDA

In [9]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 256, device=device) # directly create a tensor on GPU
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])
    print(y.to("cpu", torch.double)[0:5, 0:5])

tensor([[0.2608, 0.1572, 0.2389, 0.0016, 0.9399],
        [0.5568, 0.0984, 0.4221, 0.7044, 0.9644],
        [0.9272, 0.4899, 0.9378, 0.0071, 0.7858],
        [0.6075, 0.4217, 0.3743, 0.3490, 0.0173],
        [0.1765, 0.3963, 0.9382, 0.5887, 0.4214]], device='cuda:0')
tensor([[0.2608, 0.1572, 0.2389, 0.0016, 0.9399],
        [0.5568, 0.0984, 0.4221, 0.7044, 0.9644],
        [0.9272, 0.4899, 0.9378, 0.0071, 0.7858],
        [0.6075, 0.4217, 0.3743, 0.3490, 0.0173],
        [0.1765, 0.3963, 0.9382, 0.5887, 0.4214]], dtype=torch.float64)


In [10]:
%%timeit
# Calculate the projection matrix of x on the GPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

6.96 ms ± 24.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Exhaustive Testing on GPU

In [11]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 10, device=device) # directly create a tensor on GPU

In [12]:
if torch.cuda.is_available():
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])

tensor([[0.3585, 0.7632, 0.5213, 0.3327, 0.2046],
        [0.4346, 0.6956, 0.1903, 0.2573, 0.8348],
        [0.6216, 0.6143, 0.0070, 0.4170, 0.1825],
        [0.5957, 0.6494, 0.0480, 0.6889, 0.5424],
        [0.0859, 0.5120, 0.9854, 0.5357, 0.0543]], device='cuda:0')


In [13]:
if torch.cuda.is_available():
    # Here is the memory of the GPU a border. 
    # A matrix with 100000 lines requires 37 GB, but only 8 GB are available.
    H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

In [14]:
if torch.cuda.is_available():
    print(H[0:5, 0:5])

tensor([[ 5.0483e-04,  3.9707e-05,  1.8646e-04, -1.6392e-04,  1.4807e-04],
        [ 3.9707e-05,  6.7378e-04,  2.1153e-04,  4.4458e-04, -1.2641e-04],
        [ 1.8646e-04,  2.1153e-04,  5.3275e-04,  2.4127e-04, -2.7844e-04],
        [-1.6392e-04,  4.4458e-04,  2.4127e-04,  9.0672e-04,  8.4044e-05],
        [ 1.4807e-04, -1.2641e-04, -2.7844e-04,  8.4045e-05,  1.2198e-03]],
       device='cuda:0')


In [15]:
if torch.cuda.is_available():
    # This operation is difficult, as an symmetric matrix is transferred 
    # back to the CPU. Is possible up to 30000 rows.
    print(H.to("cpu", torch.double)[0:5, 0:5])

tensor([[ 5.0483e-04,  3.9707e-05,  1.8646e-04, -1.6392e-04,  1.4807e-04],
        [ 3.9707e-05,  6.7378e-04,  2.1153e-04,  4.4458e-04, -1.2641e-04],
        [ 1.8646e-04,  2.1153e-04,  5.3275e-04,  2.4127e-04, -2.7844e-04],
        [-1.6392e-04,  4.4458e-04,  2.4127e-04,  9.0672e-04,  8.4044e-05],
        [ 1.4807e-04, -1.2641e-04, -2.7844e-04,  8.4045e-05,  1.2198e-03]],
       dtype=torch.float64)
